### 6.2 读写JSON数据

json模块提供了一种简单的方法来编码和解码JSON格式的数据。  
可以使用json.dump()以及json.load()来编码和解码JSON数据。

```python
# writing JSON data
with open('data.json', 'w') as f:
    json.dump(data, f)
    
# Reading data back
with open('data.json', 'r') as f:
    data = json.load(f)
```

JSON解码时会从所提供的数据中创建出字典或列表。如果想创建出其他类型的对象，可以为json.loads()方法提供object_pairs_hook或者object_hook参数。  
如下将JSON数据解码为OrderedDict，可以保持数据的顺序不变。

In [2]:
import json
s = '{"name":"ACME", "shares":50, "price":490.1}'
from collections import OrderedDict
data = json.loads(s, object_pairs_hook = OrderedDict)
print(data)

OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])


下面的代码将JSON转变为Python对象。

In [3]:
class JSONObject:
    def __init__(self, d):
        self.__dict__ = d
        
data = json.loads(s, object_hook=JSONObject)
print(data, data.name)

<__main__.JSONObject object at 0x0000020270436F60> ACME


在json.dumps()函数中使用indent参数，能使数据能够像怕print（）函数那样以漂亮的格式打印。

In [5]:
data = json.loads(s)
print(json.dumps(data, indent=4))

{
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}


In [6]:
print(json.dumps(data, sort_keys=True))

{"name": "ACME", "price": 490.1, "shares": 50}


类实例一般使无法序列化为JSON的。比如：

In [7]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
p = Point(2, 3)

In [9]:
# 会报错
# json.dumps(p)

如果像序列化类实例， 可以提供一个函数将类实例作为输入并返回一个可以被序列化处理的字典。

In [11]:
def serialize_instance(obj):
    d = {'__classname__': type(obj).__name__}
    d.update(vars(obj))
    return d

s = json.dumps(p, default=serialize_instance)

In [12]:
print(s)

{"__classname__": "Point", "x": 2, "y": 3}


In [34]:
classes = {'Point':Point}
def unserialize_object(d):
    clsname = d.pop('__classname__', None)
    if clsname:
        cls = classes[clsname]
        # Make instance without calling __init__
        obj = cls.__new__(cls) 
        for key, value in d.items():
            setattr(obj, key, value)
        return obj
    else:
        return d
a = json.loads(s, object_hook=unserialize_object)

In [35]:
a

In [36]:
vars(a)

{'x': 2, 'y': 3}

In [37]:
a.__dict__

{'x': 2, 'y': 3}